In [4]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 87.3 MB/s eta 0:00:00


In [6]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.6 MB/s eta 0:00:00


# Pembacaan Data

In [7]:
import pandas as pd
import numpy as np
import re
import nltk
import gensim
import torch

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


In [8]:
df = pd.read_csv('/content/labeled_data.csv')
df

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username,label
0,1976326435020398607,Thu Oct 09 16:38:35 +0000 2025,1,FAM ni baik bubarkan je.. lagi bagus sukan bol...,1976326435020398607,NaN,NaN,in,NaN,0,0,0,https://x.com/undefined/status/197632643502039...,202474881,NaN,negative
1,1976321981038661782,Thu Oct 09 16:20:53 +0000 2025,5,Isu pendaftaran pemain yang melanda Persatuan ...,1976321981038661782,https://pbs.twimg.com/media/G21OXPRawAAQB7B.jpg,NaN,in,NaN,0,0,1,https://x.com/undefined/status/197632198103866...,497665754,NaN,neutral
2,1976212107080462811,Thu Oct 09 15:41:03 +0000 2025,0,@PartaiSocmed FAM Malaysia aja kena skandal do...,1976311956152910130,NaN,PartaiSocmed,in,NaN,0,2,0,https://x.com/undefined/status/197631195615291...,1645925382347173888,NaN,neutral
3,1975738103169257515,Thu Oct 09 15:18:22 +0000 2025,0,@E_Blackbeard @FIFAcom @theafcdotcom @FAM_Mala...,1976306248602812738,NaN,E_Blackbeard,in,NaN,0,0,0,https://x.com/undefined/status/197630624860281...,767897318137790464,NaN,negative
4,1976286690806464548,Thu Oct 09 15:07:56 +0000 2025,0,@idextratime Lagian FAM sama TMJ ngadi2. Padah...,1976303625812180994,NaN,idextratime,in,NaN,0,0,0,https://x.com/undefined/status/197630362581218...,1280767193240592384,NaN,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,1975442072120549421,Tue Oct 07 06:04:26 +0000 2025,9,KENYATAAN MEDIA Kegagalan Pengesahan Dokumen P...,1975442072120549421,https://pbs.twimg.com/media/G2ouF6wagAAXa76.jpg,NaN,in,NaN,0,1,4,https://x.com/undefined/status/197544207212054...,339418611,NaN,NaN
1006,1975350750810349916,Tue Oct 07 06:03:23 +0000 2025,0,@Pe_Enestee @kmkzboiz @theaseanball @NepalPres...,1975441807531253787,NaN,Pe_Enestee,in,NaN,0,1,0,https://x.com/undefined/status/197544180753125...,1720665183188922368,NaN,NaN
1007,1975441770097090800,Tue Oct 07 06:03:14 +0000 2025,30,Mesyuarat pengurus pasukan dan majlis undian b...,1975441770097090800,https://pbs.twimg.com/media/G2ot0fqaYAAICsM.jpg,NaN,in,NaN,2,13,3,https://x.com/undefined/status/197544177009709...,3191632944,NaN,NaN
1008,1975440898910806224,Tue Oct 07 05:59:46 +0000 2025,0,PETALING JAYA 7 Okt Hannah Yeoh selaku Menteri...,1975440898910806224,https://pbs.twimg.com/media/G2otBVqbIAImmja.jpg,NaN,in,NaN,0,0,0,https://x.com/undefined/status/197544089891080...,1923654992722001922,NaN,NaN


In [10]:
df_labeled = df.dropna(subset=['label']).sample(250, random_state=42)

df_labeled

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username,label
151,1975923415656645030,Wed Oct 08 13:57:07 +0000 2025,0,Declaring the relevant complaint ungrouded me...,1975923415656645030,NaN,NaN,in,NaN,0,0,0,https://x.com/undefined/status/197592341565664...,1026057296,NaN,negative
6,1976296700433055897,Thu Oct 09 14:40:25 +0000 2025,0,Kakitangan tu tak sebodoh tu @FAM_Malaysia Mal...,1976296700433055897,NaN,NaN,in,NaN,0,0,0,https://x.com/undefined/status/197629670043305...,1421689117302083590,NaN,positive
97,1976083436424385014,Thu Oct 09 00:32:59 +0000 2025,6,Nak beli kepercayaan dan sokongan masyarakat b...,1976083436424385014,NaN,NaN,in,NaN,0,0,2,https://x.com/undefined/status/197608343642438...,161829569,NaN,neutral
60,1976121950138171681,Thu Oct 09 05:01:16 +0000 2025,4,@MelGohCNA @saifnasution Masalahnya are their ...,1976150952337080365,NaN,MelGohCNA,in,NaN,0,0,0,https://x.com/undefined/status/197615095233708...,2288097228,NaN,neutral
112,1975317624780431362,Wed Oct 08 16:52:07 +0000 2025,0,@wallpassjournal @FAM_Malaysia @OfficialJohor ...,1975967454628377066,NaN,wallpassjournal,in,NaN,0,0,0,https://x.com/undefined/status/197596745462837...,332964795,NaN,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,1976033423153332366,Wed Oct 08 21:14:15 +0000 2025,0,Banned je FAM b* dan bolasepak Malaysia. Selag...,1976033423153332366,NaN,NaN,in,NaN,0,0,0,https://x.com/undefined/status/197603342315333...,1427585695418183686,NaN,negative
14,1976275066607526357,Thu Oct 09 13:14:27 +0000 2025,0,Masa zaman TMJ jadi presiden FAM tak pernah pu...,1976275066607526357,https://pbs.twimg.com/media/G20jsU1b0AAbF-B.jpg,NaN,in,NaN,0,2,0,https://x.com/undefined/status/197627506660752...,410021007,NaN,positive
92,1976087534259331544,Thu Oct 09 01:43:37 +0000 2025,0,@malaysia_nt @FAM_Malaysia Boleh main ke ni. K...,1976101209766494665,NaN,malaysia_nt,in,NaN,0,0,0,https://x.com/undefined/status/197610120976649...,82904748,NaN,neutral
205,1975556410814730716,Wed Oct 08 08:55:41 +0000 2025,0,@FajarKNatanael @SkySportsNews FIFA menuduh As...,1975847554136039514,NaN,FajarKNatanael,in,NaN,0,0,0,https://x.com/undefined/status/197584755413603...,1720665183188922368,NaN,negative


# Preproccesing

In [11]:
nltk.download('wordnet')
nltk.download('stopwords')

stop_words = set(stopwords.words('indonesian'))
lemmatizer = WordNetLemmatizer()
stemmer = StemmerFactory().create_stemmer()

def clean_text(text):
    text = re.sub(r"[^a-zA-Z ]"," ", text)
    text = text.lower()
    text = " ".join([w for w in text.split() if w not in stop_words])
    text = stemmer.stem(text)
    text = " ".join([lemmatizer.lemmatize(w) for w in text.split()])
    return text

df_labeled["clean_text"] = df_labeled["full_text"].astype(str).apply(clean_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Spliting Data

In [12]:
X = df_labeled["clean_text"]
y = df_labeled["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Percobaan 1 : TF-IDF

In [13]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

models = {
    "Naive Bayes": MultinomialNB(),
    "SVM": SVC(),
    "Logistic Regression": LogisticRegression(max_iter=2000)
}

print("=== TF-IDF RESULTS ===\n")

for name, model in models.items():
    model.fit(X_train_tfidf, y_train)
    pred = model.predict(X_test_tfidf)
    print(f"\n{name}")
    print(classification_report(y_test, pred))


=== TF-IDF RESULTS ===


Naive Bayes
              precision    recall  f1-score   support

    negative       0.44      0.78      0.56        18
     neutral       0.00      0.00      0.00        12
    positive       0.61      0.55      0.58        20

    accuracy                           0.50        50
   macro avg       0.35      0.44      0.38        50
weighted avg       0.40      0.50      0.43        50


SVM
              precision    recall  f1-score   support

    negative       0.48      0.72      0.58        18
     neutral       0.00      0.00      0.00        12
    positive       0.61      0.70      0.65        20

    accuracy                           0.54        50
   macro avg       0.36      0.47      0.41        50
weighted avg       0.42      0.54      0.47        50


Logistic Regression
              precision    recall  f1-score   support

    negative       0.50      0.78      0.61        18
     neutral       0.00      0.00      0.00        12
    positive

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/m

# Percobaan 2 : Bag of Words (BOW)

In [14]:
bow = CountVectorizer()
X_train_bow = bow.fit_transform(X_train)
X_test_bow = bow.transform(X_test)

print("\n=== BAG OF WORDS RESULTS ===\n")

for name, model in models.items():
    model.fit(X_train_bow, y_train)
    pred = model.predict(X_test_bow)
    print(f"\n{name}")
    print(classification_report(y_test, pred))



=== BAG OF WORDS RESULTS ===


Naive Bayes
              precision    recall  f1-score   support

    negative       0.44      0.67      0.53        18
     neutral       0.33      0.08      0.13        12
    positive       0.55      0.55      0.55        20

    accuracy                           0.48        50
   macro avg       0.44      0.43      0.41        50
weighted avg       0.46      0.48      0.44        50


SVM
              precision    recall  f1-score   support

    negative       0.60      0.50      0.55        18
     neutral       0.00      0.00      0.00        12
    positive       0.49      0.85      0.62        20

    accuracy                           0.52        50
   macro avg       0.36      0.45      0.39        50
weighted avg       0.41      0.52      0.44        50


Logistic Regression
              precision    recall  f1-score   support

    negative       0.55      0.67      0.60        18
     neutral       0.33      0.08      0.13        12
    p

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Percobaan 3 : Word2Vec

In [18]:
sentences = [row.split() for row in X_train]

word2vec = gensim.models.Word2Vec(sentences, vector_size=100, window=5, min_count=1)

def vectorize(sentence):
    words = sentence.split()
    vectors = [word2vec.wv[w] for w in words if w in word2vec.wv]
    return np.mean(vectors, axis=0) if len(vectors) > 0 else np.zeros(100)

X_train_w2v = np.vstack(X_train.apply(vectorize))
X_test_w2v = np.vstack(X_test.apply(vectorize))
models_w2v = {
    "SVM": SVC(),
    "Logistic Regression": LogisticRegression(max_iter=2000)
}



In [20]:
print("\n=== WORD2VEC RESULTS ===\n")
for name, model in models_w2v.items():
    model.fit(X_train_w2v, y_train)
    pred = model.predict(X_test_w2v)
    print(f"\n{name}")
    print(classification_report(y_test, pred))




=== WORD2VEC RESULTS ===


SVM
              precision    recall  f1-score   support

    negative       0.53      0.56      0.54        18
     neutral       0.00      0.00      0.00        12
    positive       0.42      0.65      0.51        20

    accuracy                           0.46        50
   macro avg       0.32      0.40      0.35        50
weighted avg       0.36      0.46      0.40        50


Logistic Regression
              precision    recall  f1-score   support

    negative       0.36      1.00      0.53        18
     neutral       0.00      0.00      0.00        12
    positive       0.00      0.00      0.00        20

    accuracy                           0.36        50
   macro avg       0.12      0.33      0.18        50
weighted avg       0.13      0.36      0.19        50



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/m

# Percobaan 4 : GloVe Embedding

In [21]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip


--2025-12-11 09:38:19--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-12-11 09:38:19--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-12-11 09:38:19--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [22]:
glove = {}
with open("glove.6B.100d.txt", "r", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        glove[word] = vector

def glove_vectorize(sentence):
    words = sentence.split()
    vectors = [glove[w] for w in words if w in glove]
    return np.mean(vectors, axis=0) if len(vectors)>0 else np.zeros(100)

X_train_glove = np.vstack(X_train.apply(glove_vectorize))
X_test_glove = np.vstack(X_test.apply(glove_vectorize))


In [24]:
print("\n=== GLOVE RESULTS ===\n")

for name, model in models_w2v.items():
    model.fit(X_train_w2v, y_train)
    pred = model.predict(X_test_w2v)
    print(f"\n{name}")
    print(classification_report(y_test, pred))



=== GLOVE RESULTS ===


SVM
              precision    recall  f1-score   support

    negative       0.53      0.56      0.54        18
     neutral       0.00      0.00      0.00        12
    positive       0.42      0.65      0.51        20

    accuracy                           0.46        50
   macro avg       0.32      0.40      0.35        50
weighted avg       0.36      0.46      0.40        50


Logistic Regression
              precision    recall  f1-score   support

    negative       0.36      1.00      0.53        18
     neutral       0.00      0.00      0.00        12
    positive       0.00      0.00      0.00        20

    accuracy                           0.36        50
   macro avg       0.12      0.33      0.18        50
weighted avg       0.13      0.36      0.19        50



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/m